In [ ]:
import pandas as pd
import json
import numpy as np
from cleaner import *

In [ ]:
# df_train = read_json('./processed_data/preprocess_train.json')
# df_dev = read_json('./processed_data/preprocess_my_dev.json')
# df_test = read_json('./processed_data/preprocess_eval.json')
df_train = read_json('./original_data/train.json')
df_dev = read_json('./processed_data/my_dev.json')
df_test = read_json('./original_data/eval.json')
print("Number of training data: {}".format(df_train.shape[0]))
print("Number of developing data: {}".format(df_dev.shape[0]))
print("Number of testing data: {}".format(df_test.shape[0]))

In [ ]:
category = {
    'fake': 0,
    'real': 1
}
df_train['label'] = df_train['label'].map(category)
df_dev['label'] = df_dev['label'].map(category)

In [ ]:
# df_train[df_train['label'] == 0]['idx'].value_counts()
# for each in df_train[df_train['idx'] == 21934].values:
#     print(each[4])
df_train[df_train['idx'] == 21934]

In [ ]:
df_train[df_train['label'] == 1]['context_idx'].value_counts(), df_dev[df_dev['label'] == 1]['context_idx'].value_counts()

In [ ]:
df_train['label'].value_counts(), df_dev['label'].value_counts()

In [ ]:
# df_my_gold = pd.read_csv('./my_gold_dev.csv')
# df_dev['label'] = df_my_gold['label']
# df_dev.to_json('my_dev.json', orient='records')

In [ ]:
label = []
for i in range(len(df_test)):
    if df_test['context_idx'][i] == 0:
        label.append('real')
    else:
        label.append('fake')
df_test['label'] = label
print(df_test['label'].value_counts())
df_test[['idx', 'context_idx', 'label']].to_csv('eval.csv', index=False)

In [ ]:
# same idx will contain the same label
df_predicted = pd.read_csv('my_gold_dev.csv')
df_predicted['label'].value_counts()

In [ ]:
diff, diff_with_root = 0, 0
for idx in df_predicted['idx'].unique():
    real_fake_counts = df_predicted[df_predicted['idx'] == idx]['label'].value_counts()
    real_fake_list = real_fake_counts.keys().to_list()
    if len(real_fake_list) != 1:
        # method 1: same as root
        root = df_predicted.loc[(df_predicted['idx'] == idx) & (df_predicted['context_idx'] == 0), 'label'].tolist()[0]

        # method 2: same as majority
        majority = real_fake_list[np.argmax(real_fake_counts)]

        df_predicted.loc[(df_predicted['idx'] == idx), 'label'] = majority
        
        if root != majority:
            diff_with_root += 1

        diff += 1
diff, diff_with_root

In [ ]:
df_predicted['label'].value_counts()
df_predicted.to_csv('dev.csv', index=False)

In [ ]:
df_train['combine'] = ['[CLS] '] + df_train['text'] + [' [SEP] '] + df_train['reply'] + [' [SEP]']

In [ ]:
df_train['combine'].to_csv(r'LM/training.txt', header=None, index=None, sep=' ')